In [ ]:
import json
import pytz

import datetime as dt
import numpy as np

from voltoolbox.fit.option_quotes import OptionSnapshot
from voltoolbox.fit.forward_fit import fit_forward_curve
from voltoolbox.fit.vol_fit import prepare_vol_quotes, TargetSlice, fit_atm_vol
from voltoolbox.calendar import nyse_calendar
from voltoolbox import BusinessTimeMeasure


file = 'vol_SPX_20210527_1857.json'
with open(file, 'r') as f:
    quotes_dict = json.loads(f.read())

quotes = OptionSnapshot.from_json_dict(quotes_dict)

BOX_SPREAD = 50.0 / 10000.0 #50bp
fitted_forwards = fit_forward_curve(quotes, BOX_SPREAD)

pricing_dt = quotes.time_stamp
if pricing_dt.tzinfo is None:
    pricing_dt = pytz.UTC.localize(pricing_dt)  

business_time = BusinessTimeMeasure(nyse_calendar(), 0.5, 252.0)

vol_quotes = {}
atm_vols = {}
for opt_sl in quotes.slices:
    forward = fitted_forwards.get(opt_sl.expiry, None)
    if forward:
        vol_sl = prepare_vol_quotes(opt_sl, forward, BOX_SPREAD, pricing_dt, business_time)
        vol_quotes[vol_sl.expiry] = vol_sl

        target_sl = TargetSlice.create(vol_sl)
        atm_vols[vol_sl.expiry] = fit_atm_vol(target_sl)


In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = np.array([6.0, 4.0]) * 2

plt.plot(fitted_forwards.keys(), fitted_forwards.values(), marker='+')
plt.grid()

slice_index_w = widgets.IntSlider(value=0, min=0, max =len(vol_quotes)-1)
@widgets.interact(slice_index=slice_index_w)
def plot_callput_parity(slice_index):      
    expi_dt = list(vol_quotes)[slice_index]
    vol_sl = vol_quotes[expi_dt]
    
    put = vol_sl.put
    put_mids = np.array(put.mids)
    put_errs = np.array(put.errs)
    plt.plot(put.log_moneyness, put_mids - put_errs, marker='^', linestyle='none', color='blue', markersize=4)
    plt.plot(put.log_moneyness, put_mids + put_errs, marker='v', linestyle='none', color='blue', markersize=4)

    call = vol_sl.call
    call_mids = np.array(call.mids)
    call_errs = np.array(call.errs)
    plt.plot(call.log_moneyness, call_mids - call_errs, marker='^', linestyle='none', color='red', markersize=4)
    plt.plot(call.log_moneyness, call_mids + call_errs, marker='v', linestyle='none', color='red', markersize=4)

    atm_vol, err = atm_vols[expi_dt]
    plt.axhline(y=atm_vol, color = 'black')

    plt.xlim((-0.60, 0.40))
    plt.ylim((0.0, 0.60))
    plt.axvline(x=0.0, color = 'black')
        
    plt.title(f'{vol_sl.symbol}  {vol_sl.expiry.date()}    fwd={round(vol_sl.forward, 2)}  atm vol={round(100* atm_vol, 2)} ')
    plt.grid()
